# Motivation
论文原文：[《Deep Interest Network for Click-Through Rate Prediction》](https://arxiv.org/abs/1706.06978)

参考学习地址：
* [详解 Deep Interest Network: 为阿里创造更多广告收入](https://zhuanlan.zhihu.com/p/54085498)
* [从DIN到DIEN看阿里CTR算法的进化脉络](https://zhuanlan.zhihu.com/p/78365283)

动机是 **<font color="red">从用户丰富的历史行为中捕捉用户兴趣</font>**，从而提高广告CTR。文中指出用户兴趣有如下两个特点：
* 多样性（Diversity），即用户在线上购物时往往同时表现出多种兴趣
* 局部聚焦（Local Activation），即用户是否点击推荐的商品往往基于之前的部分兴趣，而非所有兴趣。例如：一个喜欢游泳与零食的用户，推荐给他泳镜，与他之前是否买过泳衣有关，而与之前买过薯片等行为无关。
受Attention机制（不同特征赋予不同weight，某些weight高的特征主导本次预测）启发，DIN针对当前候选广告局部地激活用户的历史兴趣，赋予和候选广告相关的历史兴趣更高的weight，从而实现**local activation**，而weight的多样性同时也实现了用户兴趣的多样性表达。

## Feature Representation
CTR预估中通常为稀疏特征，包含one-hot和multi-hot类型：

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/din-features.png" width = "400" height = "400" alt="DIN特征表示" align=center />

 ## NetWork

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/din.png" width = "1000" height = "500" alt="DIN" align=center />

 ### Base Model（Embedding&MLP）
 DIN的基础网络结构不复杂，主要注意embedding的部分，对one-hot特征做embedding，对multi-hot特征（对应User Behaviors部分）做embedding后再pooling（<font color="red">由于不同客户序列长度不同，pooling可以保证embedding定长</font>）。随后拼接作为DNN的输入。

例如：一个样本$x=[t_1^T,t_2^T,...,t_M^T]^T$，其中$t_i\in R^{K_i}$代表第i个特征组有$K_i$个唯一取值。
$$
\begin{align}
\underbrace{[0,0,0,0,1,0,0]}_{weekday=Friday}\underbrace{[0,1]}_{gender=Female}\underbrace{[0,...,1,...,1,...,0]}_{visited_cate_ids={Bag,Book}}
\end{align}
$$
代表有$M=3$个特征组，分别对应$K_1=7,K_2=2,K_3=...$。对第i个特征组，另$W^i=[w_1^i,...,w_{K_i}^i]\in R^{D\times K_i}$代表第i个embedding字典，其中$w_j^i\in R^D$代表其中第j个embedding向量。嵌入操作遵循表查找机制：
* 如果是One-Hot向量，当第$j$个元素为1时，$t_i$的embedding表征对应一维向量$e_i=w_j^i$。如左图中user profile features，candidate ad，context features都是one-hot，每个点对应一个特征组$t_i$，每个特征组都是一个一维向量；
* 如果时Multi-Hot向量，当$t_i[j]=1, \text{where } j\in \{j_1,j_2,...,j_k\}$时，$t_i$的embedding表征对应多维向量${e_{j_1},e_{j_2},...,e_{j_k}}=\{w_{j_1}^i, w_{j_2}^i,...,w_{j_k}^i\}$。由于对每个样本$j_k$不固定，所以$e_i=pooling_{sum}(e_{j_1},e_{j_2},...,e_{j_k})$。如左图中user behaviors，一个Goods中包含了cate_id，shop_id和goods_id（分别对应三个颜色点），如果把每个Goods中的同颜色点取出，对应一个特征组$t_i$。可以理解成，这里把每个命中的商品，所对应的特征组（cate，shop，goods）embedding都取出来，然后按照同一个商品三个特征组concat，也即concat后一个Goods对应的维度为$(1, M_{multi}\times D)$，对一个样本而言为$(j_k, M_{multi}\times D)$，在0处Sum Pooling降维得到固定$(1, M_{multi}\times D)$

所以Embedding（含Pooling）后总维度为$(1, M_{one}\times D + M_{multi}\times D)$

 ### Activation Unit
 DIN的关键点在于AU的设计，通过计算候选广告与用户最近N个历史行为商品的相关性权重weight，将其作为加权系数来对这N个行为商品的embeddings做sum pooling（也即得到了用户兴趣）。AU通过weight的多样化实现了更Diverse的用户兴趣表达。而 weight 是根据候选广告与历史行为一起决定的，即使用户的历史行为相同，但不同的候选广告与各个行为的weight也是不同的，即用户兴趣表示也是不同的。<font color="red">DIN希望通过 Activation Unit 实现“pay attention”，即赋予和候选广告相关的历史兴趣更高的weight，从而实现Local Activation。</font>

 AU内部是一个简单的多层网络，输入时候选广告的Embedding、历史行为商品的Embedding，以及两者的外积。例如针对一个历史行为商品Goods：
 * 假设一个候选广告对应$M_{ad}$个特征组，那么输入为$v_A\in R^{(1, M_{ad} \times D)}$
 * 当前Goods对应$M_{goods}$个特征组，原文中为3（cate_id，shop_id，goods_id），输入为$e_j\in R^{(1, M_{goods} \times D)}$
 * 在Base Model中，对于一个候选者，不论候选广告是什么，每个goods都是等权的（因为sum pooling是goods维等权降维）。为了实现和候选广告相关的历史商品权重更大（i.e. local activation），除了扩展embedding size（但容易导致训练加重和过拟合）可以缓解外，DIN使用的是weighted sum pooling：
$$
\begin{align}
v_U(A)=f(v_A,e_1,e_2,...,e_H)=\Sigma_{j=1}^Ha(e_j,v_A)e_j=\Sigma_{j=1}^Hw_je_j
\end{align}
$$
其中$a(e_j,v_A)=W_2^T\cdot ReLU(W_1\cdot [e_j,v_A,e_j\odot v_A] + b_1) + b_2$。注意，这里的$e_j\odot v_A$代表的是哈达玛积（见Wide&Deep中，或xDeepFM中举例）。所以前置要求为<font color="red"> *$M_{ad}=M_{goods}$*，由于是两两embeddingg对应位置的交互，本质是也是语义对齐的，对应了xDeepFM中vector-wise的交互。</font>

 ### Training Tech
 文中提出了两个技巧来辅助训练
 * Adaptive Regularization / Mini-Batch Aware Regularization（MAR）。文中提出了自适应正则，即每次mini-batch，只在非0特征对应参数上计算L2正则（针对特征稀疏性），且正则强度与特征频次有关，频次越高正则强度越低，反之越高（针对特征长尾效应）。例如，在第m次mini-batch训练，对第 j 个特征的embedding向量$w_j$的梯度更新。其中$B_m$表示第m次mini-batch的数据集，$n_j$表示特征j在整个训练集S中出现的频次，$\alpha_{mj}\in \{0,1\}$表示$B_m$中是否至少有一个样本的特征j取值为1，$\lambda =0.01$为正则化参数。
  $$
  \begin{align}
  w_j \leftarrow w_j - \eta[\frac{1}{|B_m|}\Sigma_{(x,y)\in B_m}\frac{\partial L(p(x,y))}{\partial w_j}+\lambda \frac{\alpha_{mj}}{n_j}w_j]
  \end{align}
  $$
 * Data Adaptive Activation Function（Dice）。Dice的主要动机是随数据分布变化动态地调整rectified poinit（限定在了数据均值$E[s]$）。PReLU是常见的激活函数，其固定折点为0，原文将PReLU改进，将曲线中间平滑表示，平滑方式也与数据分布有关。
$$
\begin{align}
f_{PReLU}(s) = 
\begin{cases}
    s, & \text{if } x > 0 \\
    \alpha s,      & \text{if } x \le 0
\end{cases}
= p(s)\cdot s + (1-p(s))\cdot \alpha s \text{ where } p(s)=I(s>0) 为indicator function
\end{align}
$$

$$
\begin{align}
f_{Dice}(s) = 
p(s)\cdot s + (1-p(s))\cdot \alpha s & \text{, where } p(s)=\frac{1}{1+e^{-\frac{s-E[s]}{\sqrt{Var[s]+\epsilon }}}}
\end{align}
$$

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/din-dice.png" width = "600" height = "300" alt="Dice" align=center />